In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
data = pd.read_csv(r'C:\Users\Elijah\Downloads\breast_cancer.csv')
data = data.drop('Unnamed: 32', axis=1).sort_values(by='diagnosis')

In [3]:
a = [range(2, 11), range(12, 21), range(22, 32)] # 2-11, 12-21, 22-32
X = data.iloc[:, a[2]] # средние значения данных, стандартная ошибка, худшее значение
y = data['diagnosis']

In [4]:
y = pd.get_dummies(y.to_frame(), drop_first=True)

In [5]:
for col in X.columns:
    X[col] = pd.cut(X[col], 30)

<ipython-input-5-ef4ce8cadee8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = pd.cut(X[col], 30)
<ipython-input-5-ef4ce8cadee8>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = Encoder_X.fit_transform(X[col])
<ipython-input-5-ef4ce8cadee8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [6]:
X = pd.get_dummies(X, columns=X.columns, drop_first=True)

In [7]:
# данные готовы (значения '?' остались нетронутыми)

In [8]:
# внутренний код

import numpy.random as rnd

def Init_Net(df_input, df_output, K):
    I=len(df_input.columns)
    O=len(df_output.columns)
    d=I-O
    delta=d//(K+1)
    l=[I] 
    for i in range(K):
        l.append(I-(i+1)*delta)
    l.append(O)
    LW=[]
    for i in range(K+1):
        W=rnd.uniform(0,0.1,(l[i],l[i+1]))
        LW.append(W)
    return LW

def Sigmoid(S,a):
    return 1/(1+np.exp(-a*S))

def Work_Item(X,W,a):
    S=np.dot(X,W)
    Y=Sigmoid(S,a)
    return Y

def Work(X,LW,a):
    K=len(LW)
    LY=[X]
    for i in range(K):
        Y=Work_Item(X,LW[i],a)
        LY.append(Y)
        X=Y
    return LY

def Correction(LY,LW,D,eta,a):
    K=len(LW)
    for k in list(range(K+1))[::-1][:-1]:
        if k==K:
            error=t=D-LY[k]
        else:
            t=np.dot(Delta,LW[k].T)
        Delta=a*t*LY[k]*(1-LY[k])
        LW[k-1]+=eta*np.dot(LY[k-1].T,Delta)
    return LW, error

def Train(df_input, df_output, KIntra, a, eta, MaxIter, epsilon):
    I=len(df_input.columns)
    O=len(df_output.columns)
    LW=Init_Net(df_input, df_output, KIntra)
    Iter=0
    N=len(df_input.index)
    Success=True
    StopCondition=False
    while not StopCondition:
        ErrorEnergy=[]
        for i in range(N):
            X=np.array(df_input.iloc[i])
            X.shape=(1,I)
            D=np.array(df_output.iloc[i])
            D.shape=(1,O)
            LY=Work(X,LW,a)
            LW,error=Correction(LY,LW,D,eta,a)
            ErrorEnergy.append(0.5*np.sum(error**2))
        AvgErrorEnergy=np.mean(np.array(ErrorEnergy))
        Iter+=1
        StopCondition=(AvgErrorEnergy<epsilon)
        if Iter>MaxIter:
            Success=False
            break
    return LW, Success, Iter
    
def Test(df_input,df_output,KIntra,a,LW,epsilon):
    I=len(df_input.columns)
    O=len(df_output.columns)
    N=len(df_input.index)
    K=len(LW)
    KError=0
    for i in range(N):
        X=np.array(df_input.iloc[i])
        X.shape=(1,I)
        D=np.array(df_output.iloc[i])
        D.shape=(1,O)
        LY=Work(X,LW,a)
        Y=LY[K]
        error=D-Y
        if 0.5*np.sum(error**2)>=epsilon*5:
            KError+=1
    return KError

In [9]:
from sklearn.model_selection import train_test_split

MaxIter=500 #int(input('maximum number of iterations: '))
KIntra=1 #int(input('number of intermediate layers: '))
eta=0.3 #float(input('learning rate: '))
epsilon=0.01 #float(input('epsilon: '))
a=1 #float(input('parameter a: '))
df_train_input, df_test_input, df_train_output, df_test_output = train_test_split(X, y, test_size=0.2, random_state=0)
LW, Success, Iter = Train(df_train_input, df_train_output, KIntra, a, eta, MaxIter, epsilon)
if Success:
    print('fine')
else:
    print('unsuccessful learning')

fine


In [10]:
print(Iter, 'iterations')
KError = Test(df_test_input, df_test_output, KIntra, a, LW, epsilon)
print(KError, 'errors')

10 iterations
10 errors
